Импортируем нужные библиотеки

In [1]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.loggers import MLFlowLogger

Импортируем необходимые классы

In [2]:
import config
from dataset import DataModule
from model import RankingModel

Задаем сиды, чтобы можно было воспроизводить результаты

In [3]:
torch.random.manual_seed(config.RANDOM_SEED)
pl.seed_everything(config.RANDOM_SEED)

Seed set to 42


42

In [4]:
loggers = []

Подключаем `TensorBoardLogger` логирование

In [5]:
if config.TENSORBOARD_LOGGER:
    tensorboard_logger = TensorBoardLogger(save_dir="tb_logs", name="ranking_model")
    loggers.append(tensorboard_logger)

Подключаем `MLFlow`

In [6]:
if config.MLFLOW_LOGGER:
    mlf_logger = MLFlowLogger(
        experiment_name=config.EXPERIMENT_NAME,
        tracking_uri=config.TRACKING_URL,
        log_model=config.LOG_MODEL,
    )
    loggers.append(mlf_logger)

Создаем необходимые инстанты: данные, модель, учитель.

In [7]:
dm = DataModule(
    config.TRAIN_DATA_PATH,
    config.TEST_DATA_PATH,
    config.BATCH_SIZE,
    config.NUM_WORKERS,
    config.TRAIN_VAL_RATIO,
)
model = RankingModel(config.INPUT_SIZE, config.LEARNING_RATE)
trainer = pl.Trainer(
    accelerator=config.ACCELERATOR,
    devices=config.DEVICES,
    min_epochs=config.MIN_EPOCHS,
    max_epochs=config.MAX_EPOCHS,
    logger=loggers,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Обучаем модель, и тестируем и валидируем.

In [8]:
trainer.fit(model, dm)
trainer.validate(model, dm)
trainer.test(model, dm)


  | Name    | Type    | Params
------------------------------------
0 | fc1     | Linear  | 3.8 K 
1 | fc2     | Linear  | 32.9 K
2 | fc3     | Linear  | 129   
3 | loss_fn | BCELoss | 0     
------------------------------------
36.9 K    Trainable params
0         Non-trainable params
36.9 K    Total params
0.147     Total estimated model params size (MB)


Epoch 12: 100%|██████████| 9/9 [00:00<00:00, 126.96it/s]                   

`Trainer.fit` stopped: `max_epochs=13` reached.


Validation DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 80.91it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss            0.6278309226036072
        val_ndcg            0.8412059545516968
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Testing DataLoader 0: 100%|██████████| 24/24 [00:00<00:00, 359.09it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.5728029012680054
        test_ndcg      

[{'test_loss': 0.5728029012680054, 'test_ndcg': 0.5769645571708679}]

Сохраняем модель, чтобы использовать её в проде

In [9]:
script = model.to_torchscript()
torch.jit.save(script, config.MODEL_PATH)

Проверим размер модели

In [10]:
print("model size: {:.3f}MB".format(model.get_model_size()))

model size: 0.141MB


В целом модель занимает мало места, можно не делать квантизацию или пост-пруннинг.